## Quickdraw con EMNIST features

En esta notebook tomé 5 clases de la base de datos de quickdraw: apple, smiley face, rabbit, bed, sun y pencil. Después dividí la red en dos partes, la parte de feature layers y la parte de clasification. Copié los pesos de la red entrenada con la base de datos de EMNIST, dicha red tenia la misma arquitectura. Posteriormente reentrené la red para obtener los pesos de las últimas capas (las de clasificación), y obtuve una predicción del 97.38%, lo cual es satisfactorio.

In [4]:
from keras.models import *
from keras.callbacks import *
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import keras
import numpy as np
from keras.models import load_model
#import cv2
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt
from module import printim

In [5]:
apple=np.load('apple.npy')
smiley_face=np.load('smiley_face.npy')
rabbit=np.load('rabbit.npy')
pencil=np.load('pencil.npy')
bed=np.load('bed.npy')
num_classes=5

In [6]:
model_EMNIST=load_model('modelo_EMNIST.h5')
num_classes=5

TypeError: ('Keyword argument not understood:', 'data_format')

In [4]:
m_model=Sequential()
feature_layers = [
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape = (28,28,1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    Flatten(),
]
classification_layers = [
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
]
for l in feature_layers:
    l.trainable = False
m_model=Sequential(feature_layers+classification_layers)
m_model.summary()

TypeError: softmax() got an unexpected keyword argument 'axis'

In [16]:
weights= [layer.get_weights() for layer in model_EMNIST.layers]
for i in np.arange(0,len(feature_layers),1):
        m_model.layers[i].set_weights(weights[i])
m_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])


In [18]:
muestras=20000
x_train=np.vstack([apple[0:muestras,:],pencil[0:muestras,:],rabbit[0:muestras,:],smiley_face[0:muestras,:],bed[0:muestras,:]])
#print(x_train.shape)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')/255
y_train=np.array([np.zeros(muestras),np.ones(muestras),np.ones(muestras)*2,np.ones(muestras)*3,np.ones(muestras)*4])
y_train=y_train.reshape(muestras*5)
y_train = keras.utils.to_categorical(y_train, num_classes)
m_model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1)

Epoch 1/10
100000/100000 [==============================] - 86s 858us/step - loss: 0.1889 - acc: 0.9413
Epoch 2/10
100000/100000 [==============================] - 85s 852us/step - loss: 0.1245 - acc: 0.9621
Epoch 3/10
100000/100000 [==============================] - 85s 850us/step - loss: 0.1072 - acc: 0.9673
Epoch 4/10
100000/100000 [==============================] - 86s 861us/step - loss: 0.0971 - acc: 0.9700
Epoch 5/10
100000/100000 [==============================] - 85s 848us/step - loss: 0.0880 - acc: 0.9723
Epoch 6/10
100000/100000 [==============================] - 84s 844us/step - loss: 0.0826 - acc: 0.9741
Epoch 7/10
100000/100000 [==============================] - 84s 842us/step - loss: 0.0760 - acc: 0.9761
Epoch 8/10
100000/100000 [==============================] - 85s 845us/step - loss: 0.0716 - acc: 0.9782
Epoch 9/10
100000/100000 [==============================] - 84s 843us/step - loss: 0.0694 - acc: 0.9787
Epoch 10/10
100000/100000 [==============================] - 84s

In [66]:
x_test=np.vstack([apple[muestras:int(muestras+muestras*0.3),:],pencil[muestras:int(muestras+muestras*0.3),:],rabbit[muestras:int(muestras+muestras*0.3),:],smiley_face[muestras:int(muestras+muestras*0.3),:],bed[muestras:int(muestras+muestras*0.3),:]])
#print(x_train.shape)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')/255
y_test=np.array([np.zeros(int(muestras*0.3)),np.ones(int(muestras*0.3)),np.ones(int(muestras*0.3))*2,np.ones(int(muestras*0.3))*3,np.ones(int(muestras*0.3))*4])
y_test=y_test.reshape(int(muestras*0.3)*5)
prediction=m_model.predict_classes(x_test)
#print('{:.2f}'.format(np.where(prediction-y_test==0)[0].shape[0]*100/y_test.shape[0])+'%')
np.savetxt('prediction.txt',prediction)

97.38%
[0 0 0 ... 4 4 4]
